### Import Libraries

Import necessary libraries for database connection, data manipulation, and visualization.


In [1]:
# Importing Libraries
import sqlite3
import pandas as pd

import ipywidgets as widgets
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display


### Connect to the Database

Establish a connection to the local SQLite database `telecom.db`.


In [2]:
# Creating connection to the local database
conn = sqlite3.connect("telecom.db")

### Query and Display Sample Data

Run SQL queries to fetch sample data from various tables for exploration.


In [3]:
# Query and Display to learn more about data and what information can be used to further analyse
query1 = """

SELECT * FROM dim_cities
LIMIT 5;

"""

query2 = """

SELECT * FROM dim_date
LIMIT 5;

"""

query3 = """

SELECT * FROM dim_plan
LIMIT 5;

"""

query4 = """

SELECT * FROM fact_market_share
LIMIT 5;

"""

query5 = """

SELECT * FROM fact_plan_revenue
LIMIT 5;

"""

df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)
df3 = pd.read_sql_query(query3, conn)
df4 = pd.read_sql_query(query4, conn)
df5 = pd.read_sql_query(query5, conn)

### Query: Highest Revenue City

Find the city generating the highest total revenue from telecom plans.


In [4]:
#1 "Which cities generate the highest total revenue from telecom plans?"

high_rev_city_query = """
SELECT city_name, SUM(R.plan_revenue_crores) AS total_revenue
    FROM dim_cities C LEFT JOIN fact_plan_revenue R 
    ON  C.city_code = R.city_code
    GROUP BY city_name
    ORDER BY total_revenue DESC 
    LIMIT 1
"""

high_rev_city_df = pd.read_sql_query(high_rev_city_query, conn)
high_rev_city_df

,city_name,total_revenue
0,Mumbai,280.17


### Query: Revenue Before vs After 5G Rollout

Analyze how total plan revenue changed before and after the 5G rollout.


In [5]:
#2 "How did total plan revenue change before vs after the 5G rollout?"

before_after_5g_query = """
SELECT "before/after_5g", SUM(R.plan_revenue_crores) AS total_revenue
    FROM dim_date D LEFT JOIN fact_plan_revenue R
    ON D.date = R.date
    GROUP BY "before/after_5g"
"""

before_after_5g_df = pd.read_sql_query(before_after_5g_query, conn)
before_after_5g_df

,before/after_5g,total_revenue
0,After 5G,1068.38
1,Before 5G,882.37


### Query: Top Market Share by City

Identify the company with the highest average market share in each city.


In [6]:
#3 “Which company has the highest average market share in each city?”
top_ms_per_city_query = """
WITH rank_ms AS (
    SELECT 
        C.city_name, 
        MS.company, 
        AVG(MS.ms_pct) AS avg_market_share,
        RANK() OVER (PARTITION BY C.city_name ORDER BY AVG(MS.ms_pct) DESC) AS company_rank
    FROM fact_market_share MS LEFT JOIN dim_cities C 
    ON MS.city_code = C.city_code
    GROUP BY C.city_name, MS.company

)
SELECT city_name, company, avg_market_share
FROM rank_ms
WHERE company_rank = 1;
"""

top_ms_per_city_df = pd.read_sql_query(top_ms_per_city_query, conn)
top_ms_per_city_df

,city_name,company,avg_market_share
0,Ahmedabad,PIO,36.55125
1,Bangalore,PIO,33.79625
2,Chandigarh,PIO,36.14500
3,Chennai,PIO,35.45500
4,Coimbatore,PIO,35.26750
5,Delhi,PIO,34.98250
6,Gurgaon,PIO,35.19375
7,Hyderabad,PIO,35.48625
8,Jaipur,PIO,34.96875
9,Kolkata,PIO,37.31500


### Query: High Revenue and Market Share Companies

Find companies with high average market share and revenue after the 5G rollout.


In [7]:
#4 "Find company with the high average market share and revenue"
high_rev_ms_query = """
WITH revenue_per_city AS (
    SELECT C.city_name, 
    AVG(R.plan_revenue_crores) AS avg_revenue
    FROM fact_plan_revenue R
    JOIN dim_cities C ON R.city_code = C.city_code
    JOIN dim_date D ON R.date = D.date
    WHERE D."before/after_5g" = "After 5G" 
    GROUP BY C.city_name
),

market_share_pc AS (
    SELECT C.city_name,
    MS.company,
    AVG(MS.ms_pct) AS avg_ms
    FROM fact_market_share MS
    JOIN dim_cities C ON MS.city_code = C.city_code
    JOIN dim_date D ON MS.date = D.date
    WHERE D."before/after_5g" = "After 5G" 
    GROUP BY C.city_name, MS.company
)

SELECT M.city_name,
M.company,
M.avg_ms,
R.avg_revenue
FROM market_share_pc M JOIN revenue_per_city R ON M.city_name = R.city_name
WHERE M.avg_ms > 30 AND R.avg_revenue > 3
ORDER BY R.avg_revenue DESC, M.avg_ms DESC;
"""

high_rev_ms_df = pd.read_sql_query(high_rev_ms_query, conn)
high_rev_ms_df

,city_name,company,avg_ms,avg_revenue
0,Mumbai,PIO,33.8625,3.98575
1,Delhi,PIO,35.2775,3.31350
2,Delhi,Britel,31.0425,3.31350
3,Kolkata,PIO,37.6125,3.24075


### Query: Aggregate Data for Dashboard

Fetch and preprocess data for dashboard visualizations.


In [8]:
# Import the database int
import sqlite3
import pandas as pd


# connect to database
conn = sqlite3.connect("telecom.db")

# Query the average of before and after 5g period
query = """
SELECT 
    D."before/after_5g" AS period,
    C.city_name,
    FMS.company,
    FPR.plan_revenue_crores,
    FMS.ms_pct,
    D.time_period
FROM fact_plan_revenue FPR
JOIN dim_cities C ON FPR.city_code = C.city_code
JOIN dim_date D ON FPR.date = D.date
JOIN fact_market_share FMS ON FPR.city_code = FMS.city_code AND FPR.date = FMS.date;
"""

df = pd.read_sql_query(query, conn)

df["plan_revenue_crores"] = pd.to_numeric(df["plan_revenue_crores"], errors="coerce")
df["ms_pct"] = pd.to_numeric(df["ms_pct"], errors="coerce")
df.dtypes

period                  object
city_name               object
company                 object
plan_revenue_crores    float64
ms_pct                 float64
time_period             object
dtype: object

### Grouped Data: Before vs After 5G

Aggregate data by period for analysis.


In [9]:
before_after_5g = df.groupby(["period"]).agg({
    "plan_revenue_crores": "mean",
    "ms_pct": "mean"
}).reset_index()
before_after_5g

,period,plan_revenue_crores,ms_pct
0,After 5G,1.780633,20.0001
1,Before 5G,1.470617,20.0000


### Grouped Data: Before vs After 5G by City

Aggregate data by period and city for visualization.


In [10]:
before_after_5g_city = df.groupby(["period","city_name"]).agg({
    "plan_revenue_crores": "mean",
    "ms_pct": "mean"
}).reset_index()
before_after_5g_city

,period,city_name,plan_revenue_crores,ms_pct
0,After 5G,Ahmedabad,1.59775,20.0015
1,After 5G,Bangalore,2.81975,20.0005
2,After 5G,Chandigarh,0.50475,20.0000
3,After 5G,Chennai,2.50600,20.0000
4,After 5G,Coimbatore,0.77350,20.0000
5,After 5G,Delhi,3.31350,20.0005
6,After 5G,Gurgaon,0.45800,20.0000
7,After 5G,Hyderabad,2.00650,20.0000
8,After 5G,Jaipur,1.15625,19.9990
9,After 5G,Kolkata,3.24075,20.0005


### Visualization: Revenue Before vs After 5G (Bar Chart)

Create a grouped bar chart to compare revenue before and after 5G rollout by city.


In [11]:
# Revenue of before and after installation of 5G for each city
before_after_5g_city["period"] = pd.Categorical(before_after_5g_city["period"], categories=["Before 5G", "After 5G"], ordered=True)

# Grouped bar chart
fig_bar = px.bar(
    before_after_5g_city,
    x="city_name",
    y="plan_revenue_crores",
    color="period",
    barmode="group",
    text_auto=".2f",
    title="Plan Revenue Before vs After 5G by City",
    color_discrete_map={
        "Before 5G": "#1f77b4",  # Blue
        "After 5G": "#e74c3c"    # Red
    }
)

_ = fig_bar.update_layout(
    xaxis_title="City",
    yaxis_title="Revenue (Crores)",
    xaxis_tickangle=-45,
)

In [12]:
before_after_5g_period = df.groupby(["period","time_period"]).agg({
    "plan_revenue_crores": "mean",
    "ms_pct": "mean"
}).reset_index()

### Visualization: KPI Indicators

Create KPI indicators for average revenue and revenue change after 5G rollout.


In [13]:
# Average of both periods
avg_revenue_total = before_after_5g["plan_revenue_crores"].mean()

# Extract specific values
before = before_after_5g.loc[before_after_5g["period"] == "Before 5G", "plan_revenue_crores"].values[0]
after = before_after_5g.loc[before_after_5g["period"] == "After 5G", "plan_revenue_crores"].values[0]

# --- KPI 1: Average Revenue ---
kpi_avg = go.Figure(go.Indicator(
    mode="number",
    value=avg_revenue_total * 10,
    title={"text": "Avg Revenue (All Periods)"},
    number={"valueformat": ".2f", "suffix": " M"}
))

# --- KPI 2: Delta After vs Before ---
kpi_delta = go.Figure(go.Indicator(
    mode="number+delta",
    value=after * 10,
    delta={
        "reference": before * 10,
        "relative": True,
        "valueformat": ".2%",
        "increasing": {"color": "green"},
        "decreasing": {"color": "red"}
    },
    title={"text": "After vs Before 5G"},
    number={"valueformat": ".2f", "suffix": " M"}
))

kpi_avg.update_layout(
    font=dict(
        size=20,
    ),
    height=300,
    title="5G Telecom Dashboard"
)

kpi_delta.update_layout(
    font=dict(
        size=20,
    ),
    height=300,
    title="5G Telecom Dashboard"
)


In [14]:
# query the average revenue for each month for plotting line graph.

query = """
SELECT 
    COUNT(FPR.plans),
    FPR.plan_revenue_crores,
    D.time_period,
    D.month_name,
    D."before/after_5g" AS period
FROM fact_plan_revenue FPR
JOIN dim_date D ON FPR.date = D.date
GROUP BY month_name
ORDER BY CASE D.month_name
    WHEN 'Jan' THEN 1
    WHEN 'Feb' THEN 2
    WHEN 'Mar' THEN 3
    WHEN 'Apr' THEN 4
    WHEN 'May' THEN 5
    WHEN 'Jun' THEN 6
    WHEN 'Jul' THEN 7
    WHEN 'Aug' THEN 8
    WHEN 'Sep' THEN 9
    WHEN 'Oct' THEN 10
    WHEN 'Nov' THEN 11
    WHEN 'Dec' THEN 12
END
"""

df = pd.read_sql_query(query, conn)
df["plan_revenue_crores"] = pd.to_numeric(df["plan_revenue_crores"], errors="coerce")

fig_line = px.line(
    df,
    x="month_name",
    y="plan_revenue_crores",
    markers= True,
    title="Average Plan Revenue per Month",
    color_discrete_map={
        "Before 5G": "#1f77b4",  # Blue
        "After 5G": "#e74c3c"    # Red
    }
)


In [15]:
fig_line_ba5g = px.line(
    df,
    x="time_period",
    y="plan_revenue_crores",
    color="period",
    markers=True,
    title="Average Plan Revenue per Month",
    color_discrete_map={
        "Before 5G": "#1f77b4",  # Blue
        "After 5G": "#e74c3c"    # Red
    }
)


### Visualization: Interactive Revenue Filter by City and Period


In [16]:
# Make a dropdown list to display the filter for different cities.
city_dropdown = widgets.Dropdown(
    options=["All"] + sorted(before_after_5g_city["city_name"].unique().tolist()),
    description="City:"
)

period_dropdown = widgets.Dropdown(
    options=["All", "Before 5G", "After 5G"],
    description="Period:"
)

In [17]:
def update_plot(city, period):
    filtered = before_after_5g_city.copy()
    
    if city != "All":
        filtered = filtered[filtered["city_name"] == city]
        
    if period != "All":
        filtered = filtered[filtered["period"] == period]
        
    fig = px.bar(
        filtered,
        x="city_name",
        y="plan_revenue_crores",
        color="period",
        barmode="group",
        title=f"Revenue by City ({period if period != 'All' else 'All Periods'})",
            color_discrete_map={
        "Before 5G": "#1f77b4",  # Blue
        "After 5G": "#e74c3c"    # Red
    }
    )
    fig.show()


ui = widgets.HBox([city_dropdown, period_dropdown])
interactive_plot = widgets.interactive_output(update_plot, {
    "city": city_dropdown,
    "period": period_dropdown
})

display(ui, interactive_plot)

Output()

### Combine all plots and graphs to create a mini dashboard


In [18]:
dashboard = make_subplots(
    rows=4, cols=2,
    row_heights=[0.2, 0.4, 0.4, 0.4],
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}],  # Row 1
        [{"type": "xy", "colspan": 2}, None], # Row 2: Full-width bar chart
        [{"type": "xy", "colspan": 2}, None], # Row 3: Full-width line chart
        [{"type": "xy", "colspan": 2}, None]  # Row 4: Another
    ],

)

# 🔹 Row 1: KPI Indicator
for trace in kpi_avg.data:
    dashboard.add_trace(trace, row=1, col=1)
    
for trace in kpi_delta.data:
    dashboard.add_trace(trace, row=1, col=2)

# 🔹 Row 2: Bar chart
for trace in fig_bar.data:
    dashboard.add_trace(trace, row=2, col=1)

# 🔹 Row 3: Line chart
for trace in fig_line.data:
    dashboard.add_trace(trace, row=3, col=1)
    
# 🔹 Row 4: Line chart before and after 5g
for trace in fig_line_ba5g.data:
    dashboard.add_trace(trace, row=4, col=1)
    
dashboard.update_layout(
    height=850,
    title="5G Telecom Dashboard",
    showlegend=True
)

dashboard.show()
